In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from io import StringIO
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

In [2]:
years = [2023, 2024, 2025] ## get list of years to import 
base_url = "https://www.warrennolan.com/softball/{}/sos-rpi" ## get url

In [4]:
all_years_data = []

for year in years:
    url = base_url.format(year)
    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the first table on the page (it's already nicely formatted)
    table = soup.find('table')

    # Convert the table to a dataframe
    df = pd.read_html(str(table))[0]
    df["Year"] = year  # Add year column

    all_years_data.append(df)

# Combine all years into one dataframe
sos = pd.concat(all_years_data, ignore_index=True)

# Preview the combined dataframe
print(sos.head())


/var/folders/rp/gf132r1n5bj14p__kxdf2hrm0000gp/T/ipykernel_4860/1123037262.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
/var/folders/rp/gf132r1n5bj14p__kxdf2hrm0000gp/T/ipykernel_4860/1123037262.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


            Team     SOS  Rank Opp Record  Opp Win Percent  Year
0       Stanford  0.6372     1  2215-1074           0.6731  2023
1       Oklahoma  0.6286     2  2260-1158           0.6609  2023
2        Alabama  0.6211     3  2412-1278           0.6535  2023
3            LSU  0.6151     4  2087-1155           0.6436  2023
4  Florida State  0.6151     5  2394-1298           0.6481  2023


/var/folders/rp/gf132r1n5bj14p__kxdf2hrm0000gp/T/ipykernel_4860/1123037262.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


In [5]:
sos.to_csv("sos.csv")

In [2]:
years_nonconf = list(range(2021, 2026)) ## get list of years to import 
base_url = "https://d1softball.com/nitty-gritty/?season={}" ## get url

In [3]:
# Set up headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

In [4]:
all_years_data = []

for year in years_nonconf:
    driver.get(base_url.format(year))
    time.sleep(2)

    frames = []
    while True:
        # Read whatever table is currently visible
        tables = pd.read_html(driver.page_source)
        if not tables:
            break
        for table in tables:
            frames.append(table.assign(Year=year))


        # Attempt to click "Next" or "Show More"
        try:
            nxt = driver.find_element(By.XPATH, "//a[text()='Next' or text()='›' or text()='>']")
            # If it's disabled, stop
            if "disabled" in nxt.get_attribute("class").lower():
                break
            nxt.click()
            time.sleep(2)
        except:
            break

    if frames:
        all_years_data.append(pd.concat(frames, ignore_index=True))

driver.quit()

# Combine into one DataFrame and export to HTML
nonconf = pd.concat(all_years_data, ignore_index=True)
#nonconf.to_html("nonconf_all_years.html", index=False)

print(nonconf.head())

/var/folders/rp/gf132r1n5bj14p__kxdf2hrm0000gp/T/ipykernel_7045/2795322715.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(driver.page_source)
/var/folders/rp/gf132r1n5bj14p__kxdf2hrm0000gp/T/ipykernel_7045/2795322715.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(driver.page_source)
/var/folders/rp/gf132r1n5bj14p__kxdf2hrm0000gp/T/ipykernel_7045/2795322715.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(driver.page_source)
/var/folders/rp/gf132r1n5bj14p__kxdf2hrm0000gp/T/ipykernel_7045/2795322715.py:10: FutureWarning: Passing literal ht

  Unnamed: 0_level_0 Unnamed: 1_level_0  Overall         RPI Change        \
                 RPI               Team   Record RPI SOS      1-Day 7-Day   
0                  1           Oklahoma     56-4   1  16          -     -   
1                  2            Alabama     52-9   2   4          -     -   
2                  3               UCLA     47-7   3   3          -     -   
3                  4      Florida State  49-13-1   4  11          -     -   
4                  5            Florida    45-11   5   6          -     -   

  Non-Conference         Location                 RPI Quads                 \
          Record RPI SOS     Home    Road Neutral    Quad 1 Quad 2  Quad 3   
0           37-3   5  39     18-0    18-2    20-2      23-4    7-0     7-0   
1           26-3   4  28     36-4    13-3     3-2      24-8   11-1     6-0   
2           21-3   3  11     27-2    16-2     4-3      17-7    7-0    17-0   
3           22-7   1  10     24-3  15-4-1    10-6      15-9    7-1  18

In [5]:
nonconf.to_csv("nonconf_wp.csv")